# NOSQL Enunciado Práctica

# TED Talks

TED, *Technology, Entertainment, Design*  es una organización sin ánimo de lucro dedicada a las *ideas dignas de difundir* (https://es.wikipedia.org/wiki/TED). Para muchas personas, estas charlas resultan muy inspiradoras e interesantes. TED da la oportunidad de escuchar, de manera gratuita, a conferenciantes que son referencia a nivel mundial en diversos campos.

**Objetivo de la práctica**

El objetivo de la práctica es definir un modelo de datos en Cassandra y en MongoDb para guardar las charlas de TED de forma que se puedan respondar una serie de preguntas de forma óptima.

**¿De dónde se han obtenido los datos?**

Los datos se han obtenido de Kaggle (https://www.kaggle.com/rounakbanik/ted-talks). Tal y como mencionan, los datos corresponden a todas las charlas subidas a TED.com hasta el día 21 de septiembre de 2017. 

**¿Qué información contienen? ¿y formato?**

Hay dos dataset, *TED_MAIN.csv* que contiene los metadatos de las charlas, incluidos los conferenciantes y *transcripts.csv* que contiene las transcripciones de cada charla . Están en formato *CSV* como se puede ver. Los metadatos de las charlas (campos de TED_MAIN) son:

+ Comments: número de comentarios sobre la charla.
+ description : Breve descripción de la charla.
+ Duration: Duración de la charla en segundos. 
+ event: Evento donde la charla se produjo.
+ film_date: fecha en formato UNIX timestamp
+ Languages: idiomas
+ main_speaker: conferenciante principal.
+ name: El nombre oficial.
+ num_speaker: número de conferenciantes.
+ published_date: fecha de publicación en UNIx timestamp.
+ ratings: El número de veces que la charla la han votado dentro de una lista de categorías (divertida, inspiradora,...). Es una lista/array de objetos JSON del estilo {'id': 7, 'name': 'Funny', 'count': 19645}
+ related_talks: Una lista de charlas recomendadas para ver a continuación. Es una lista de objetos JSON del estilo {'id': 865, 'hero': 'https://pe.tedcdn.com/images/ted/172559_800x600.jpg', 'speaker': 'Ken Robinson', 'title': 'Bring on the learning revolution!', 'duration': 1008, 'slug': 'sir_ken_robinson_bring_on_the_revolution', 'viewed_count': 7266103} 
+ speaker_occupation: La ocupación del conferenciante principal.
+ tags: Temas asociados con la charla.
+ Title: el título de la charla. Coincide con el "name" pero sin el nombre del conferenciante.
+ url: la url.
+ views: número de visitas.

**Carga de datos**

Los dos ficheros: ted_main.csv y transcripts.csv podrán importarse a pandas dataframes para su manipulación previa antes de cargarse en las bases de datos.

**¿Qué preguntas se desean realizar? **

Las consultas deberán resolverse en la medida de lo posible en la BD, es decir, no vale hacerlo todo en pandas.

Preguntas a responder :

+ ¿Cuál es el top 10 de las charlas más vistas?
+ ¿Cuántas charlas hay por año?
+ ¿Cuál es el top 10 de las charlas más Ingeniosas (Ingenious)? (en Cassandra, pensar una estructura que reponda a la pregunta para cualquier categoría, no solo Ingeniosas)
+ ¿Cuales son las categorías más recurrentes?
+ ¿Cuáles son las charlas más recomendadas? (las que aparecen más veces en related_talks)
+ ¿En qué charlas se habla de una palabra o expresión concreta? (en Cassandra habrá que crear un SASIIndex según se describe en el seguiente enlace https://docs.datastax.com/en/dse/5.1/cql/cql/cql_using/useSASIIndex.html)


**Modelado**

Respecto al modelo de datos adoptado en ambas BBDD, deberá dibujarse un pequeño diagrama explicativo e insertarlo en el notebook.

## Modelado Cassandra

Primero de todo, he juntado toda la información que se tiene. Una vez hecho esto, he modificado el campo "film_date" y he hecho que solo salga el año, ya que es lo que creo que es de interés para luego realizar la búsqueda por año. También creo que sería bueno que a parte del "name", el "film_date" sea una Partition Key, de modo que si tuvieramos más de un nodo, distribuiria las Ted Talks entre ellos en función del año, y posteriormente sería mucho más rápida la búsqueda de las mismas dado un año en concreto. 

<img src="Tablas_Cassandra.png",width=900,height=600>
<br><br> 

Las tablas son las siguientes:

Talks: Tabla donde estan todas las charlas, donde la clave PK es Año, por lo que he explicado arriba, y donde se encuentra el numero de visitas, las veces que ha sido recomendada (esto lo he sacado transformando los datos con anterioridad) y el nombre de la charla. 

Categoria_recurrentes: EN este caso la PK sería categoría, ya que no creo que name tenga que estar en esta tabla, ya que no le veo sentido. 

Charlas_transcripts: La primary Key Name y el texto correspondiente a la charla, donde se usará el analizador de texto con el SASSIndex. 

Ratings: Cada Rating con el numero correspondiente de veces que ha sido valorado como tal. 

He entendido por categoria el campo "Tags", aunque me hace dudar si es la de "Tags" o la de "Ratings". De todos modos, habiendo hecho las consultas tanto en Cassandra como en MongoDB, salen los mismos valores. 

## Carga de datos en pandas

In [1]:
import pandas as pd
import time

In [2]:
import sys
# sys.setdefaultencoding() does not exist, here!
reload(sys)  # Reload does the trick!
sys.setdefaultencoding('ASCII')


In [2]:
df_ted = pd.read_csv("ted_main.csv", encoding = 'utf8')
df_transcripts = pd.read_csv("transcripts.csv", encoding = 'utf8')
df_ted['film_date'] = df_ted['film_date'].apply(lambda x: time.strftime('%Y', time.localtime(x)))
df = pd.merge(df_ted, df_transcripts, on = ['url'], how = 'inner')


In [3]:
df['tags'][0]

u"['children', 'creativity', 'culture', 'dance', 'education', 'parenting', 'teaching']"

### DataFrame Valoración de Charlas

In [4]:
for i in range(0,len(df['ratings'])):
    df['ratings'][i] = eval(df['ratings'][i])

In [5]:
df['name'][0]

u'Ken Robinson: Do schools kill creativity?'

In [6]:
df_charlas_valoraciones = pd.DataFrame(columns=['Name'])
for index in range(0,len(df)):
    df_charlas_valoraciones.loc[index] = df['name'][index]

In [7]:
df_charlas_valoraciones.head(2)

,Name
0,Ken Robinson: Do schools kill creativity?
1,Al Gore: Averting the climate crisis


In [8]:
tipo_ratings=[]
for linea in range(0,len(df['ratings'])):
    for elemento in range(0,len(df['ratings'][linea])):
        tipo_ratings.append(df['ratings'][linea][elemento]['name'])

In [9]:
tipo_ratings_unicos = list(set(tipo_ratings))
print(tipo_ratings_unicos)

['Beautiful', 'Funny', 'Inspiring', 'Obnoxious', 'OK', 'Fascinating', 'Informative', 'Persuasive', 'Longwinded', 'Ingenious', 'Jaw-dropping', 'Unconvincing', 'Courageous', 'Confusing']


In [10]:
lista = []
for i in range(0,len(df['ratings'])):
    tmp_dic = {}
    for j in range(0,len(df['ratings'][i])):
        tmp_dic[df['ratings'][i][j]['name']]=df['ratings'][i][j]['count']
    lista.append(tmp_dic)

In [11]:
df_valoraciones = pd.DataFrame(columns=[tipo_ratings_unicos])

In [12]:
for i in range(0,len(lista)):    
    df_valoraciones.loc[i] = [lista[i]['Beautiful'],lista[i]['Funny'],lista[i]['Inspiring'],lista[i]['Obnoxious'],lista[i]['OK'],
                            lista[i]['Fascinating'],lista[i]['Informative'],lista[i]['Persuasive'],lista[i]['Longwinded'],
                            lista[i]['Ingenious'],lista[i]['Jaw-dropping'],lista[i]['Unconvincing'],lista[i]['Courageous'],
                            lista[i]['Confusing']]

In [13]:
ratings_and_charlas = pd.concat([df_charlas_valoraciones, df_valoraciones], axis=1, join_axes=[df_charlas_valoraciones.index])

In [14]:
print(len(df))

2467


### Trato los datos para las categorias más recurrentes

In [15]:
print(df['tags'][0])
lista_prueba = eval(df['tags'][0])

['children', 'creativity', 'culture', 'dance', 'education', 'parenting', 'teaching']


In [16]:
for i in range(0,len(df['tags'])):
    df['tags'][i] = eval(df['tags'][i])

print(type(df['tags'][0]))

<type 'list'>


In [17]:
tags1=[]
for i in range(0,len(df['tags'])):
    for elemento in range(0,len(df['tags'][i])):
        tags1.append(df['tags'][i][elemento])
        

tags1_unnique = list(set(tags1))

veces=[]
for i in range(0,len(tags1_unnique)):
        veces.append(tags1.count(tags1_unnique[i]))

In [18]:
data = {'Categorias': tags1_unnique, 'Count': veces}
dataframe_categorias_recurrentes=pd.DataFrame.from_dict(data)

### Trato los datos para las charlas más recomendadas

In [14]:
for i in range(0,len(df['related_talks'])):
    df['related_talks'][i] = eval(df['related_talks'][i])

1 - Me guardo todas las charlas que existen en un array

In [15]:
charlas = []
for indice in range(0,len(df['name'])):
        charlas.append(df['name'][indice])

2 - Me guardo en otro array todas las charlas recomendadas

In [16]:
charlas_recomendadas=[]
for indice in range(0,len(df['related_talks'])):
    for i in range(0,len(df['related_talks'][indice])):        
        #print(df['related_talks'][indice][i]['title'])
        charlas_recomendadas.append(df['related_talks'][indice][i]['speaker']+": "+df['related_talks'][indice][i]['title'])

3 - Compruebo de cada una de las charlas que hay, cuantas veces ha sido recomendada

In [17]:
charlas_recomendadas_veces=[]
for i in range(0,len(charlas)):
        charlas_recomendadas_veces.append(charlas_recomendadas.count(charlas[i]))


In [18]:
data_veces_charlas_recomendadas = {'Charla': charlas, 'Veces': charlas_recomendadas_veces}
dataframe_veces_charlas_recomendadas=pd.DataFrame.from_dict(data_veces_charlas_recomendadas)

In [37]:
print(df['name'][0])
print(df['related_talks'][0][0]['title'])
print("---------------------")
print(data_veces_charlas_recomendadas['Charla'][0])
print(len(df['related_talks']))
print((len(df['related_talks'])*6))
print(charlas_recomendadas[0])

Ken Robinson: Do schools kill creativity?
Bring on the learning revolution!
---------------------
Ken Robinson: Do schools kill creativity?
2467
14802
Ken Robinson: Bring on the learning revolution!


## Conexión con Cassandra

In [24]:
%load_ext cql

In [25]:
%cql DROP KEYSPACE tedtalks

'No results.'

In [26]:
%%cql
CREATE KEYSPACE tedtalks
WITH replication = {'class':'SimpleStrategy', 'replication_factor': 1};

'No results.'

In [27]:
%cql USE tedtalks;

'No results.'

## Creacción de las tablas Cassandra

In [28]:
%%cql
CREATE TABLE talks(
    year int,
    views int,
    vecesrecomendada int,
    name text,
    PRIMARY KEY ((year),views,vecesrecomendada,name)
    )

'No results.'

In [ ]:
%%cql
DROP TABLE talks;

In [29]:
%%cql
CREATE TABLE ratings(
    Name text,
    Beautiful int,
    Funny int,
    Inspiring int,
    Obnoxious int,
    OK int,
    Fascinating int,
    Informative int,
    Persuasive int,
    Longwinded int,
    Ingenious int,
    Jawdropping int,
    Unconvincing int,
    Courageous int,
    Confusing int,
    PRIMARY KEY (Name)
    )

'No results.'

In [30]:
%%cql
CREATE TABLE categorias_recurrentes(
    Categoria text,
    Apariciones int,
    PRIMARY KEY ((Categoria),Apariciones)
    )

'No results.'

In [31]:
%%cql
CREATE TABLE charlas_transcripts ( 
  name text, 
  transcript text,
  PRIMARY KEY ((name),transcript)
)

'No results.'

## Carga de datos en Cassandra

In [32]:
from cassandra.cluster import Cluster, BatchStatement, ConsistencyLevel
cluster = Cluster()
session = cluster.connect('tedtalks')

In [33]:
def execute_query(sql):
    rows = session.execute(sql)
    return pd.DataFrame(list(rows))

In [34]:
def insert_talks(df,df2):

    sql_insert = """
INSERT INTO talks (
year,
views,
vecesrecomendada,
name
) VALUES (%s, %s, %s, %s)
"""
    for index in range(0,len(df)):
        
        film_date=int(df.ix[index, "film_date"])
        views=int(df.ix[index, "views"])
        vecesrecomendada=int(df2.ix[index,"Veces"])
        name=df.ix[index, "name"]
        
        
        data = [ 
                film_date,
                views,
                vecesrecomendada,
                name                
            ]
    
        
        session.execute(sql_insert, data)

In [35]:
insert_talks(df,dataframe_veces_charlas_recomendadas)

In [ ]:
sql = """
SELECT name,year,views
FROM talks
"""
resultado_talks = execute_query(sql)

In [36]:
def insert_ratings(df):

    sql_insert = """
INSERT INTO ratings (
Name,
Beautiful,
Funny,
Inspiring,
Obnoxious,
OK,
Fascinating,
Informative,
Persuasive,
Longwinded,
Ingenious,
Jawdropping,
Unconvincing,
Courageous,
Confusing
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
    for index in df_valoraciones.index:
        data = [
                ratings_and_charlas.ix[index, "Name"],
                ratings_and_charlas.ix[index, "Beautiful"],
                ratings_and_charlas.ix[index, "Funny"],
                ratings_and_charlas.ix[index, "Inspiring"],
                ratings_and_charlas.ix[index, "Obnoxious"],
                ratings_and_charlas.ix[index, "OK"],
                ratings_and_charlas.ix[index, "Fascinating"],
                ratings_and_charlas.ix[index, "Informative"],
                ratings_and_charlas.ix[index, "Persuasive"],
                ratings_and_charlas.ix[index, "Longwinded"],
                ratings_and_charlas.ix[index, "Ingenious"],
                ratings_and_charlas.ix[index, "Jaw-dropping"],
                ratings_and_charlas.ix[index, "Unconvincing"],
                ratings_and_charlas.ix[index, "Courageous"],
                ratings_and_charlas.ix[index, "Confusing"]
            ]
        session.execute(sql_insert, data)

In [37]:
insert_ratings(ratings_and_charlas)

In [38]:
def insert_categories_and_counts(df):

    sql_insert = """
INSERT INTO categorias_recurrentes (
Categoria,
Apariciones
) VALUES (%s, %s)
"""
    for index in df.index:
        data = [
                df.ix[index, "Categorias"],
                df.ix[index, "Count"]
            ]
        session.execute(sql_insert, data)

In [39]:
insert_categories_and_counts(dataframe_categorias_recurrentes)

In [50]:
def insert_tabla_ssindex(df):
    sql_insert = """
INSERT INTO charlas_transcripts (
name,
transcript
) VALUES (%s, %s)
"""
    for index in df.index:
        data = [
                df.ix[index, "name"],
                df.ix[index, "transcript"]
            ]
        session.execute(sql_insert, data)

In [51]:
insert_tabla_ssindex(df)

#### Top 10 charlas más vistas

In [42]:
sql = """SELECT name, views FROM talks"""
top10_charlas_visitas = execute_query(sql)
top10_charlas_visitas.sort_values("views", ascending = False).head(10)

,name,views
855,Ken Robinson: Do schools kill creativity?,47227110
1113,Amy Cuddy: Your body language may shape who yo...,43155405
488,Simon Sinek: How great leaders inspire action,34309432
746,Brené Brown: The power of vulnerability,31168150
487,Mary Roach: 10 things you didn't know about or...,22270883
2383,Julian Treasure: How to speak so that people w...,21594632
2463,Jill Bolte Taylor: My stroke of insight,21190883
854,Tony Robbins: Why we do what we do,20685401
1566,James Veitch: This is what happens when you re...,20475972
1112,Cameron Russell: Looks aren't everything. Beli...,19787465


#### Charlas por año

In [43]:
sql = """SELECT year, count(*) as NUM FROM talks GROUP BY year """
charlas_por_year = execute_query(sql)
charlas_por_year.sort_values('year', ascending = False)

,year,num
7,2017,98
18,2016,244
10,2015,239
0,2014,237
19,2013,268
6,2012,258
15,2011,239
3,2010,258
2,2009,219
20,2008,80


#### 10 charlas más ingeniosas

In [44]:
sql = """
SELECT name, Ingenious
FROM ratings
"""
resultado_ratings = execute_query(sql)
resultado_ratings.sort_values('ingenious', ascending = False).head(10)

,name,ingenious
2321,Ken Robinson: Do schools kill creativity?,6073
1329,Pranav Mistry: The thrilling potential of Sixt...,4743
466,Simon Sinek: How great leaders inspire action,3413
1836,Johnny Lee: Free or cheap Wii Remote hacks,3249
591,Hans Rosling: The best stats you've ever seen,3202
855,Pattie Maes + Pranav Mistry: Meet the SixthSen...,2366
433,Sal Khan: Let's use video to reinvent education,1971
1335,Tim Urban: Inside the mind of a master procras...,1957
1953,"Arthur Benjamin: A performance of ""Mathemagic""",1944
1633,James Veitch: This is what happens when you re...,1766


#### Categorias más recurrentes

In [45]:
sql = """
SELECT Categoria, Apariciones FROM categorias_recurrentes """
resultado_categorias_recurrentes = execute_query(sql)
resultado_categorias_recurrentes.sort_values('apariciones', ascending = False).head(10)

,categoria,apariciones
56,technology,712
207,science,548
199,global issues,490
204,culture,476
17,TEDx,418
120,design,408
373,business,338
310,entertainment,287
14,health,234
43,innovation,224


#### Charlas más recomendadas

In [46]:
sql = """SELECT name,vecesrecomendada FROM talks """
resultado_charlas_mas_recomendadas = execute_query(sql)
resultado_charlas_mas_recomendadas.sort_values('vecesrecomendada', ascending = False).head(10)

,name,vecesrecomendada
1204,Anne Lamott: 12 truths I learned from life and...,38
1871,Dan Gilbert: The surprising science of happiness,33
855,Ken Robinson: Do schools kill creativity?,25
796,Janine Benyus: Biomimicry's surprising lessons...,25
1760,Aaron Koblin: Visualizing ourselves ... with c...,25
1133,"David Whyte: A lyrical bridge between past, pr...",25
419,Sylvia Earle: My wish: Protect our oceans,25
2056,Ari Wallach: 3 ways to plan for the (very) lon...,24
266,Wade Davis: Dreams from endangered cultures,24
836,Alan Russell: The potential of regenerative me...,24


#### En qué charlas de habla de una palabra o expresión concreta

In [ ]:
%%cql
CREATE CUSTOM INDEX stdanalyzer_idx ON charlas_transcripts (transcript) USING 'org.apache.cassandra.index.sasi.SASIIndex'
WITH OPTIONS = {
'mode': 'CONTAINS',
'analyzer_class': 'org.apache.cassandra.index.sasi.analyzer.StandardAnalyzer',
'analyzed': 'true'
};

In [53]:
sql = """SELECT * FROM charlas_transcripts WHERE transcript LIKE 'science' """
resultado_transcript_sassindex = execute_query(sql)
resultado_transcript_sassindex

,name,transcript
0,Avi Rubin: All your devices can be hacked,"I'm a computer science professor, and my area ..."
1,Kristina Gjerde: Making law on the high seas,Today I'm going to take you on a voyage to som...
2,Peter Donnelly: How juries are fooled by stati...,"As other speakers have said, it's a rather dau..."
3,Kevin B. Jones: Why curiosity is the key to sc...,Science. The very word for many of you conjure...
4,Linda Liukas: A delightful way to teach kids a...,Code is the next universal language. In the se...
5,Nick Veasey: Exposing the invisible,"So, 120 years ago, Dr. Röntgen X-rayed his wif..."
6,Matthieu Ricard: The habits of happiness,"So, I guess it is a result of globalization th..."
7,Amy Cuddy: Your body language may shape who yo...,So I want to start by offering you a free no-t...
8,Garrett Lisi: An 8-dimensional model of the un...,"Whoa, dude.(Laughter)Check out those killer eq..."
9,Sangeeta Bhatia: This tiny particle could roam...,In the space that used to house one transistor...
